In [ ]:
import numpy as np
import json
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from lac.localization.imu_recovery import estimate_imu_odometry
from lac.utils.frames import invert_transform_mat
from lac.utils.plotting import plot_poses
from lac.util import load_data, compute_odometry_sigmas
from lac.params import DT

%load_ext autoreload
%autoreload 2

In [ ]:
# data_path = "../../output/DataCollectionAgent/double_loop_preset1"
data_path = "/home/shared/data_raw/LAC/runs/triangles_preset6"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

In [ ]:
gt_odometry = []
for i in range(len(poses) - 1):
    gt_odometry.append(np.linalg.inv(poses[i]) @ poses[i + 1])

In [ ]:
len(imu_data), len(poses)

## Direct odometry estimation


In [ ]:
# imu_data[k] is measurement for time window between pose[k] to pose[k+1]

imu_odometry = []

for k in range(len(imu_data)):
    a = imu_data[k][:3]
    omega = imu_data[k][3:]
    R_curr = poses[k + 1][:3, :3]
    if k == 0:
        v_prev = np.zeros(3)
    else:
        v_prev = (poses[k][:3, 3] - poses[k - 1][:3, 3]) / DT
    odom = estimate_imu_odometry(a, omega, R_curr, v_prev)
    imu_odometry.append(odom)

In [ ]:
est_pose = initial_pose
est_poses = [est_pose]

for k in range(len(imu_odometry)):
    est_pose = est_pose @ imu_odometry[k]
    est_poses.append(est_pose)

In [ ]:
fig = plot_poses(poses[:1000], color="black", no_axes=True, name="Ground Truth")
fig = plot_poses(est_poses[:1000], fig=fig, color="orange", no_axes=True, name="SLAM")
fig.show()

## Integration and differencing


In [ ]:
from lac.localization.imu_recovery import ImuEstimator

In [ ]:
# imu_data[k] is measurement for time window between pose[k] to pose[k+1]

imu_estimator = ImuEstimator(initial_pose)

est_pose = initial_pose
est_poses = [est_pose]
imu_odometry = []

for k in range(2000):
    # Ground truth for first 1000 steps
    if k < 1000:
        odom = gt_odometry[k].copy()
        est_pose = est_pose @ odom
        imu_estimator.update_pose(est_pose)
    elif k < 1001:
        odom = gt_odometry[k].copy()
        odom[:3, 3] += np.array([0, 0, 1e-5])
        est_pose = est_pose @ odom
        imu_estimator.update_pose(est_pose)
    # Switch to IMU for 1000
    else:
        imu_estimator.update(imu_data[k])
        odom = imu_estimator.get_pose_delta()
        est_pose = est_pose @ odom

    est_poses.append(est_pose)

In [ ]:
fig = plot_poses(poses[:2000], color="black", no_axes=True, name="Ground Truth")
fig = plot_poses(est_poses, fig=fig, color="orange", no_axes=True, name="SLAM")
fig.show()

## Every image frame


In [ ]:
# imu_data[k] is measurement for time window between pose[k] to pose[k+1]

imu_estimator = ImuEstimator(initial_pose)

imu_measurements = []
est_pose = initial_pose
est_poses = [est_pose]
eval_odoms = []
imu_odoms = []

for k in range(len(imu_data)):
    imu_measurements.append(imu_data[k])
    if k % 2 == 0 and k != 0:  # image available
        for measurement in imu_measurements:
            imu_estimator.update(measurement)
        imu_measurements.clear()

        gt_odom = np.linalg.inv(poses[k - 2]) @ poses[k]

        odom = np.linalg.inv(est_pose) @ imu_estimator.get_pose()
        est_pose = est_pose @ odom
        est_poses.append(est_pose)

        imu_odoms.append(odom)
        eval_odoms.append(gt_odom)

In [ ]:
sigma_rotation, sigma_translation = compute_odometry_sigmas(imu_odoms, eval_odoms)
print(f"Rotation sigma (rad): {sigma_rotation}")
print(f"Translation sigma (m): {sigma_translation}")

In [ ]:
fig = plot_poses(poses[:2000], color="black", no_axes=True, name="Ground Truth")
fig = plot_poses(est_poses, fig=fig, color="orange", no_axes=True, name="SLAM")
fig.show()

### Mixing


In [ ]:
# imu_data[k] is measurement for time window between pose[k] to pose[k+1]

imu_estimator = ImuEstimator(initial_pose)

imu_measurements = []
est_pose = initial_pose
est_poses = [est_pose]

eval_odoms = []
imu_odoms = []

for k in range(2000):
    imu_measurements.append(imu_data[k])
    if k % 2 == 0 and k != 0:  # image available
        gt_odom = np.linalg.inv(poses[k - 2]) @ poses[k]
        # Use VO (ground truth)
        if k < 1000:
            est_pose = est_pose @ gt_odom
            est_poses.append(est_pose)
            imu_estimator.update_pose_from_vo(est_pose)
        # Use IMU
        else:
            for measurement in imu_measurements:
                imu_estimator.update(measurement)
            odom = np.linalg.inv(est_pose) @ imu_estimator.get_pose()
            est_pose = est_pose @ odom
            est_poses.append(est_pose)

            eval_odoms.append(gt_odom)
            imu_odoms.append(odom)

        est_poses.append(est_pose)
        imu_measurements.clear()

In [ ]:
fig = plot_poses(poses[:2000], color="black", no_axes=True, name="Ground Truth")
fig = plot_poses(est_poses, fig=fig, color="orange", no_axes=True, name="SLAM")
fig.show()

In [ ]:
sigma_rotation, sigma_translation = compute_odometry_sigmas(imu_odoms, eval_odoms)
print(f"Rotation sigma (rad): {sigma_rotation}")
print(f"Translation sigma (m): {sigma_translation}")